In [1]:
from utils import data_loader
from utils import model

import numpy as np

from tqdm import tqdm
from skimage.io import imread
from skimage.transform import resize
from skimage.metrics import structural_similarity, peak_signal_noise_ratio

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from utils.heat_map import HeatMap
from utils.game_metrics import GAME_metric
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
from tensorflow.keras.models import Model

2023-03-05 05:48:14.831457: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/lbbduy/anaconda3/envs/master_python/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-05 05:48:15.448522: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-03-05 05:48:15.449050: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-03-05 05:48:15.458116: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-05 05:48:15.458498: I tensorflow/core/common_runtime/gpu/gpu_device.cc:172

In [2]:
BATCH_SIZE = 1
BATCH_SAMPLE_SIZE = 64

dataset_dict = data_loader.load_dataset_paths(dataset_name='final_data', validation_split_size=0.0)

test_input_data = dataset_dict['test']['images']
test_output_data = dataset_dict['test']['density_maps']

test_dataset, test_size = data_loader.load_dataset(
    input_paths=test_input_data,
    output_paths=test_output_data,
    output_type='density_maps',
    batch_size=BATCH_SIZE,
    shuffle=False,
    downsampling_size=8,
    buffer_size=1,
    is_regression=True
)

2023-03-05 05:48:15.866007: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-03-05 05:48:15.866215: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-05 05:48:15.866606: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2023-03-05 05:48:15.866642: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-03-05 05:48:15.866656: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2023-03-05 05:48:15.866662: I tensorflow/stream_executor/plat

In [3]:
test_size

1301

In [5]:
model_regression_pretrained = model.load_pretrained_model('./model_checkpoints/model_CSR-Net-Regression-Freeze-Desity-Map_checkpoint_03_04_2023_235118.h5')

dm_layer = model_regression_pretrained.get_layer('density_map').output

model_dm_pretrained= Model(inputs=model_regression_pretrained.input, outputs=dm_layer)

In [6]:
model_dm_pretrained.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_image_input (InputLaye [(None, 480, 640, 3)]     0         
_________________________________________________________________
lambda (Lambda)              (None, 480, 640, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 480, 640, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 480, 640, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 240, 320, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 240, 320, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 240, 320, 128)     1475

In [7]:
estimation_pred = []
estimation_true = []
ssim_scores = []
psnr_scores = []
GAME_0 = []
GAME_1 = []
GAME_2 = []
GAME_3 = []

for _, data in tqdm(
        enumerate(test_dataset),
        total=len(test_dataset)
):
    count = np.squeeze(model_regression_pretrained.predict(data[0])[0], axis=-1)
    dm_pred = np.squeeze(model_dm_pretrained.predict(data[0])[0], axis=-1)

    dm_true = np.squeeze(data[1][0].numpy(), axis=-1)
    # data_range=dm_pred.max() - dm_pred.min()

    # GAME_0.append(GAME_metric(dm_true, dm_pred, 0))
    # GAME_1.append(GAME_metric(dm_true, dm_pred, 1))
    # GAME_2.append(GAME_metric(dm_true, dm_pred, 2))
    # GAME_3.append(GAME_metric(dm_true, dm_pred, 3))
    # ssim_scores.append(structural_similarity(dm_true, dm_pred, data_range=data_range))
    # psnr_scores.append(peak_signal_noise_ratio(dm_true, dm_pred, data_range=data_range))
    estimation_true.append(np.sum(dm_true))
    estimation_pred.append((np.sum(dm_pred) + np.sum(count)) / 2)

2023-03-05 05:49:31.407936: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-03-05 05:49:31.408236: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3686400000 Hz
  0%|          | 0/1301 [00:00<?, ?it/s]2023-03-05 05:49:31.538580: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2023-03-05 05:49:31.754111: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2023-03-05 05:49:31.756393: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2023-03-05 05:49:33.161004: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2023-03-05 05:49:33.199552: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256

In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
mse = mean_squared_error(estimation_true, estimation_pred)
mae = mean_absolute_error(estimation_true, estimation_pred)
# game0 = np.mean(GAME_0)
# game1 = np.mean(GAME_1)
# game2 = np.mean(GAME_2)
# game3 = np.mean(GAME_3)

# print('GAME 0:{} GAME 1:{} GAME 2:{} GAME 3:{} '.format(game0, game1, game2, game3))
print('Mean Absolute Error: {}'.format(mae))
print('Mean Square Error: {}'.format(mse))
# print('SSIM: {}'.format(np.average(ssim_scores)))
# print('PSNR: {}'.format(np.average(psnr_scores)))

# model_CSRNet_MSE_BCE_Loss_checkpoint_12_08_2022_215738.h5
# GAME 0:3.774728536605835 GAME 1:4.781242121445042 GAME 2:5.7594381245196 GAME 3:7.100420445810915
# Mean Absolute Error: 3.7747225761413574
# Mean Square Error: 37.99045944213867
# SSIM: 0.8576069038151517
# PSNR: 25.686878169856836

Mean Absolute Error: 3.8950138059512733
Mean Square Error: 42.80366879209058
